In [34]:
import zipfile
from itertools import product
import datetime as dt

import geopandas as gpd
import pandas as pd
import requests
import shapely

from r5py import TransportNetwork, TravelTimeMatrixComputer, TransportMode

1. Get the extent of AoI

In [2]:
aoi = gpd.read_file("https://github.com/Urban-Analytics-Technology-Platform/demoland-web/raw/main/web/src/data/geography.json")

In [3]:
aoi_poly = aoi.to_crs(27700).unary_union

2. Get H3 grid with the data for the AoI

In [4]:
data_folder = "../../../demoland_data"

Read the full grid

In [5]:
grid = gpd.read_parquet(f"{data_folder}/h3/grid_complete.parquet")

Get a portion of the grid covering AoI.

In [6]:
grid_aoi = grid.iloc[grid.sindex.query(aoi_poly, predicate="intersects")]

3. Make predictive models ready

    - to be done once the final models are settled.

5. Make accessibility ready
    6. Get GTFS
  
Go to https://data.bus-data.dft.gov.uk/downloads/, register and download timetable data for your region in GTFS data format.

In [7]:
gtfs_data_file = f"{data_folder}/raw/accessibility/itm_north_east_gtfs.zip"

7. Get network from OSM

Download a fresh OSM snapshot for England.

In [14]:
r = requests.get('http://download.geofabrik.de/europe/united-kingdom/england-latest.osm.pbf')
with open("england-latest.osm.pbf", "wb") as f:
    f.write(r.content)

Extract the AoI. We need a GeoJSON of the area.

In [8]:
aoi.dissolve().to_file("aoi.geojson")

And then can use osmium to get an extract.

In [16]:
!osmium extract -p aoi.geojson england-latest.osm.pbf -o aoi.osm.pbf

[======================================================================] 100% 


8. Get OS Greenspace

In [30]:
r = requests.get('https://api.os.uk/downloads/v1/products/OpenGreenspace/downloads?area=GB&format=GeoPackage&redirect')
with open("opgrsp_gpkg_gb.zip", "wb") as f:
    f.write(r.content)

Read the file.

In [9]:
with zipfile.ZipFile('opgrsp_gpkg_gb.zip', 'r') as zip_ref:
    with zip_ref.open("Data/opgrsp_gb.gpkg") as gsp:
        f = gsp.read()
        greenspace_sites = gpd.read_file(f, engine="pyogrio", layer="greenspace_site")
        greenspace_access = gpd.read_file(f, engine="pyogrio", layer="access_point")

/home/martin/mambaforge/envs/shap/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: File /vsimem/e27ecc0b3d7047ce9b7df4c11a0f0c2c has GPKG application_id, but non conformant file extension
  result = ogr_read(
/home/martin/mambaforge/envs/shap/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: File /vsimem/18929c422c094f15838bdfad8bd472fc has GPKG application_id, but non conformant file extension
  result = ogr_read(


Extract the AoI

In [10]:
greenspace_sites_aoi = greenspace_sites.iloc[greenspace_sites.sindex.query(aoi_poly, predicate="intersects")]
greenspace_access_aoi = greenspace_access.iloc[greenspace_access.sindex.query(aoi_poly, predicate="intersects")]

9. Process OS Greenspace

In [11]:
greenspace_sites_select = greenspace_sites_aoi.query(
    "function!='Allotments Or Community Growing Spaces' & function!='Golf Course' & function!='Bowling Green'"
)
publicpark = greenspace_sites_select.query("function=='Public Park Or Garden'")
playingfield = greenspace_sites_select.query("function=='Playing Field'")
othersport = greenspace_sites_select.query("function=='Other Sports Facility'")
therest = greenspace_sites_select.query(
    "function!='Playing Field' & function!='Public Park Or Garden' & function!='Other Sports Facility'"
)

In [12]:
# find 'therest' not included in the upper categories
# we use sjoin to performe a spatial filter of 'therest' polygons contained in upper categories
join11 = gpd.sjoin(therest, othersport, predicate="within", how="inner")
join12 = gpd.sjoin(therest, playingfield, predicate="within", how="inner")
join13 = gpd.sjoin(therest, publicpark, predicate="within", how="inner")

# generate list of the IDs of 'therest' contained in upper categories, in order to eliminate the corresponding polygons from the layer
list_for_diff11 = join11["id_left"].drop_duplicates().to_list()

diff11 = therest[
    ~therest.id.isin(list_for_diff11)
]  # 1st difference layer # note the negation character ~ to take the polygons NOT included

list_for_diff12 = join12["id_left"].drop_duplicates().to_list()
diff12 = diff11[~diff11.id.isin(list_for_diff12)]  # 2nd difference layer

list_for_diff13 = join13["id_left"].drop_duplicates().to_list()
diff13 = diff12[
    ~diff12.id.isin(list_for_diff13)
]  # 3rd difference layer, this is for 'therest' categories

In [13]:
# we repeat the same operation for subsequent categories:
# find 'othersport' not included in the upper categories
join21 = gpd.sjoin(othersport, playingfield, predicate="within", how="inner")
join22 = gpd.sjoin(othersport, publicpark, predicate="within", how="inner")

list_for_diff21 = join21["id_left"].drop_duplicates().to_list()
diff21 = othersport[~othersport.id.isin(list_for_diff21)]

list_for_diff22 = join22["id_left"].drop_duplicates().to_list()
diff22 = diff21[~diff21.id.isin(list_for_diff22)]  # 'othersport' difference

In [14]:
# find 'playing fields' not included in the upper categories (and viceversa?)
join31 = gpd.sjoin(playingfield, publicpark, predicate="within", how="inner")
join32 = gpd.sjoin(
    publicpark, playingfield, predicate="within", how="inner"
)  ## check it is not empty ... it is empty, we do not use this join

list_for_diff31 = join31["id_left"].drop_duplicates().to_list()
diff31 = playingfield[
    ~playingfield.id.isin(list_for_diff31)
]  # 'playingfield' difference

In [15]:
# put together all the differences layers: (and should bring out to the desired output)
together1 = pd.concat([diff13, diff22]).pipe(
    gpd.GeoDataFrame
)  # 'therest' + 'othersport' differences
together1.head()
together2 = pd.concat([together1, diff31]).pipe(
    gpd.GeoDataFrame
)  # last gdf + 'playingfield' difference
together_again = gpd.GeoDataFrame(pd.concat([together2, publicpark]), crs=27700)  # last gdf + all the public parks)

In [16]:
list_gs_id = together_again["id"].to_list()
accesspoints_edge = greenspace_access_aoi[greenspace_access_aoi.ref_to_greenspace_site.isin(list_gs_id)]
accesspoints_edge = accesspoints_edge.to_crs(27700)

together_again["area_m2"] = together_again["geometry"].area

together_again.to_file("greenspace.gpkg", layer="sites")
accesspoints_edge.to_file("greenspace.gpkg", layer="access_points")

10. Create traveltime matrix (origins are cells, destinations are cells plus greenspace entrances)

In [19]:
origins = grid_aoi.set_geometry(grid_aoi.centroid).to_crs(4326)
origins["id"] = origins.index

In [24]:
destinations = pd.concat(
    [
        origins[["id", "geometry"]],
        accesspoints_edge[["id", "geometry", "ref_to_greenspace_site"]].to_crs(4326),
    ],
    ignore_index=True
)

In [26]:
transport_network = TransportNetwork("aoi.osm.pbf", [gtfs_data_file])

generate dataframe with all from_id and all to_id pairs

In [62]:
prod = product(origins["id"].unique(), destinations["id"].unique())
empty_ttm = pd.DataFrame(prod)
empty_ttm.columns = ["from_id", "to_id"]

In [63]:
# defining variables
date_time = "2023,11,23,9,30"  # CHOOSE BEST DATE/TIME
max_time = dt.timedelta(seconds=900) # SET TO 15 MIN
walking_speed = 4.8
cycling_speed = 16

dataframe to match legmode and transitmode objects (to be inputted in the ttm computer)

In [64]:
modes_lut = pd.DataFrame(
    [
        ["transit", TransportMode.CAR, TransportMode.WALK],
        ["car", "", TransportMode.CAR],
        ["bicycle", "", TransportMode.BICYCLE],
        ["walk", "", TransportMode.WALK],
    ],
    columns=("Mode", "Transit_mode", "Leg_mode"),
)

In [65]:
# function to generate custom list of transit+transport mode for the parameter transport_modes in TravelTimeMatrixComputer
def list_making(s, z):
    if s:
        return [s] + [z]
    return [z]

In [66]:
ttm_complete = empty_ttm.copy()

# loop to compute a ttm for all the modes and generate one single ttm table in output
for row in modes_lut.itertuples():
    start_time = dt.datetime.now()
    mode = row.Mode
    transit_mode = row.Transit_mode
    leg_mode = row.Leg_mode
    transport_mode = list_making(
        transit_mode, leg_mode
    )  # creating list of objects for transport_modes parameter

    print(
        "The current mode is:",
        mode,
        ", transit is:",
        transit_mode,
        ", transport var is:",
        transport_mode,
    )
    ttm_computer = TravelTimeMatrixComputer(
        transport_network,
        origins=origins,
        destinations=destinations,
        departure=dt.datetime.strptime(date_time, "%Y,%m,%d,%H,%M"),
        max_time = max_time,
        speed_walking=walking_speed,
        speed_cycling=cycling_speed,
        transport_modes=transport_mode,
    )

    ttm = ttm_computer.compute_travel_times()
    ttm = ttm.rename(
        columns={"travel_time": f"time_{mode}"}
    )  # renaming 'travel_time' column (automatically generated) to 'time_{mode of transport}'
    #  merging the empty table generated before (with all possible origins and destinations) with the ttm, per each mode adding a travel time column
    ttm_complete = ttm_complete.merge(
        ttm, how="outer", left_on=["from_id", "to_id"], right_on=["from_id", "to_id"]
    )

    print("finished calculating ttm for mode", mode)
    end_time = dt.datetime.now()
    print("Duration for", mode, ": {}".format(end_time - start_time))

The current mode is: transit , transit is: TransportMode.CAR , transport var is: [<TransportMode.CAR: 'CAR'>, <TransportMode.WALK: 'WALK'>]
finished calculating ttm for mode transit
Duration for transit : 0:09:07.266543
The current mode is: car , transit is:  , transport var is: [<TransportMode.CAR: 'CAR'>]
finished calculating ttm for mode car
Duration for car : 0:08:44.851404
The current mode is: bicycle , transit is:  , transport var is: [<TransportMode.BICYCLE: 'BICYCLE'>]
finished calculating ttm for mode bicycle
Duration for bicycle : 0:09:46.095740
The current mode is: walk , transit is:  , transport var is: [<TransportMode.WALK: 'WALK'>]
finished calculating ttm for mode walk
Duration for walk : 0:10:16.975145


In [67]:
ttm_complete.to_parquet(f"ttm_complete.parquet")

In [68]:
ttm_complete

,from_id,to_id,time_transit,time_car,time_bicycle,time_walk
0,8919465366bffff,8919465366bffff,0.0,4.0,5.0,0.0
1,8919465366bffff,89194653477ffff,16.0,16.0,NaN,NaN
2,8919465366bffff,8919465343bffff,16.0,16.0,NaN,NaN
3,8919465366bffff,8919465340fffff,14.0,14.0,NaN,NaN
4,8919465366bffff,89194653433ffff,15.0,15.0,NaN,NaN
...,...,...,...,...,...,...
66781104,89197331847ffff,C8E183F0-C82B-4201-A71C-958A5A955BBB,NaN,NaN,NaN,NaN
66781105,89197331847ffff,D1D57228-E4C4-42A9-A214-B98878AF0D73,NaN,NaN,NaN,NaN
66781106,89197331847ffff,575DF9E9-3BB7-4ED4-9550-3D7812C75BF0,NaN,NaN,NaN,NaN
66781107,89197331847ffff,C9803849-2A52-484C-ABF7-8A0C4043FD23,NaN,NaN,NaN,NaN
